In [55]:
%matplotlib inline
from scipy.stats import hypergeom, poisson, norm
import math
import matplotlib.pyplot as plt
import pandas as pd
import re


In [2]:
M = 190000000 # Cantidad de bolas
n = 2000 # Cantidad de bolas negras
N = 100000000 # Cantidad de extracciones

def hg_pdf(k):
    return hypergeom.pmf(k, M=M, n=n, N=N )

Las cantidades de palabras en cada provincia se encuentran en el archivo cantidadesDataset.csv dentro de la carpeta tesis. 

In [107]:

df = pd.read_csv('../cantidadesDataset.csv',index_col=0)
dfcantPalabrasEnProvincia = pd.read_csv("../contrastes/provincias.csv", quotechar="\"", decimal=",",index_col='palabra')
cant = [c for c in df.columns if re.match(r'.*Palabras$', c) and (not re.match(r'.*norm', c) ) and (not re.match(r'.*rank', c) )]
palabrasEnArgentina = 192348167
def cantPalabrasTotalesEnRegion(provincias):
    res = 0
    for p in provincias:
        res += df.loc[p,'cantTotal']
    return res

def cantPalabrasWEnRegion(palabra,provincias):
    res = 0
    for p in provincias:
        res += dfcantPalabrasEnProvincia.loc[palabra, p +'Palabras']
    return res

def cantPalabraWEnArgentina(palabra):
    return dfcantPalabrasEnProvincia.loc[palabra,'cantPalabra']

def calcularPValor(palabra,region):
    M = palabrasEnArgentina # Cantidad de palabras en toda la Argentina 
    n = cantPalabraWEnArgentina(palabra) # Cantidad de palabras w en toda la Argentina 
    N = cantPalabrasTotalesEnRegion(region) # Cantidad de palabras en la region

    h0 = cantPalabrasWEnRegion('palabra',region)

    print ('Cantidad de palabras en toda la Argentina M: {0}'.format(M))
    print ('Cantidad de palabras {0} en toda la Argentina n: {1}'.format(palabra,n))
    print ('Cantidad de palabras en la región N: {0}'.format(N))
    #  hypergeom.cdf la funcion de distribucion acumulada de la hipergeometrica
    pvalue =  1 - hypergeom.cdf(h0, M, n, N) 
    print ('p-valor habiendo {0} ocurrencias de la palabra {1} en la región: {2}'.format(h0,palabra,pvalue))

## Veo el p-valor de la palabra *cha*  en Formosa y Jujuy

In [108]:

region = ['formosa','jujuy']
palabra = 'cha'
calcularPValor(palabra,region)

Cantidad de palabras en toda la Argentina M: 192348167
Cantidad de palabras cha en toda la Argentina n: 2116
Cantidad de palabras en la región N: 13136160
p-valor habiendo 1431 ocurrencias de la palabra cha en la región: 0.0


## Veo el p-valor de la palabra *pingo*  en Santiago del Estero y Tucumán

In [109]:

region = ['santiago','tucuman']
palabra = 'pingo'
calcularPValor(palabra,region)

Cantidad de palabras en toda la Argentina M: 192348167
Cantidad de palabras pingo en toda la Argentina n: 4789
Cantidad de palabras en la región N: 16594255
p-valor habiendo 1930 ocurrencias de la palabra pingo en la región: 3.30835297158e-07


In [5]:
def poisson_approx(k):
    return poisson.pmf(k, mu=(n*N)/float(M))

In [6]:
def norm_approx(k):
    p = n / float(M)
    return norm.pdf(k, loc= N*p, scale=math.sqrt(N*p*(1-p)))

# Hypergeometric mass function

In [15]:
support = xrange(N+1)

hg_masses = map(hg_pdf, support)
poisson_masses = map(hg_pdf, support)
norm_masses = map(norm_approx, support)

KeyboardInterrupt: 

En líneas generales parecen iguales... veamos más de cerca

In [ ]:
plt.stem(hg_masses[:100])

In [ ]:
plt.stem(poisson_masses[:50])

In [ ]:
plt.stem(norm_masses[:50])